In [1]:
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic_2d
%pylab inline
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product

Populating the interactive namespace from numpy and matplotlib


C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
d = ["yellow_tripdata_2016-06.csv", "yellow_tripdata_2016-05.csv", "yellow_tripdata_2016-04.csv", "yellow_tripdata_2016-03.csv", "yellow_tripdata_2016-02.csv", "yellow_tripdata_2016-01.csv"
     , "yellow_tripdata_2015-12.csv", "yellow_tripdata_2015-11.csv", "yellow_tripdata_2015-10.csv", "yellow_tripdata_2015-09.csv", "yellow_tripdata_2015-08.csv", "yellow_tripdata_2015-07.csv"
    , "yellow_tripdata_2015-06.csv"]

In [3]:
reg= pd.read_csv("regions.csv", delimiter=';')

## Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки, ошибка модели за май составила: 26.320869760849764.

In [4]:
dat= pd.read_csv('dat_c6w5-0.csv', sep=';')
dat['tim1'] = dat['tim1'].astype(datetime64)
dat.set_index(['zone', 'tim1'], inplace = True) #'Unnamed: 0'

In [5]:
#Перечень рабочих зон.
zone = [1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132,
       1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182,
       1183, 1184, 1221, 1222, 1223, 1224, 1225, 1227, 1228, 1229, 1230,
       1231, 1232, 1233, 1234, 1235, 1272, 1273, 1274, 1278, 1279, 1280,
       1281, 1282, 1283, 1284, 1285, 1286, 1287, 1326, 1327, 1331, 1332,
       1333, 1334, 1335, 1336, 1337, 1338, 1339, 1376, 1377, 1378, 1380,
       1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 1390, 1426, 1431,
       1434, 1435, 1436, 1437, 1438, 1439, 1441, 1442, 1480, 1482, 1483,
       1530, 1532, 1533, 1580, 1630, 1684, 1733, 1734, 1783, 2068, 2069,
       2118, 2119, 2168]

In [50]:
best_xgb_params = [0.1, 75, 0.01, 0.01] #лучшие параметры прошлой недели

In [139]:
%%time
# Создаем мега цикл, который посчитает качество модели и остатки в мае из прошлой работы.
xgb_params = {'learning_rate': [0.1], 'n_estimators': [75], 
              'reg_alpha': [0.01], 'reg_lambda': [0.01]}
params = list(product(xgb_params['learning_rate'], xgb_params['n_estimators'], 
                      xgb_params['reg_alpha'], xgb_params['reg_lambda']))
X_test = dat_test.copy()
X_test.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train = dat_train.copy()
X_train.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)

best_xgb_cv = None
#best_xgb_score = float('inf')
for j in range(len(params)):
    Q_xgb_1 = 0
    param = params[j]
    xgb_cv = []
    xgb_model = xgb.XGBRegressor(learning_rate=param[0], n_estimators=param[1], 
                                    reg_lambda=param[2], reg_alpha=param[3], n_jobs=-1)
    for i in range(1,7):
        Q_may = 0. # ошибка в мае
        Y_test = dat_test[str(i) + '_ans']
        Y_train = dat_train[str(i) + '_ans']
        
        
        
        #Считаем ошибку по линейной регрессии
        stud = xgb_model.fit(X_train, Y_train)
    
    
        Y_pred = stud.predict(X_test)
        Y_pred[Y_pred < 0] = 0  # All low values set to 0
        Y_score = stud.score(X_test, Y_test)
        Q_may += sum(abs(Y_test-Y_pred))
        Q_xgb_1 += Q_may
        print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_may))
        print(' ')
    
print ('Ошибка модели XGB за май: '+str(1/(102*739*6.)*Q_xgb_1))


[16:54:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 1 = 0.9737094252166182, ошибка= 3.4009436882556363
 
[16:58:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 2 = 0.9560627428551045, ошибка= 4.268252452085932
 
[17:00:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 3 = 0.9499453538072568, ошибка= 4.532657351393422
 
[17:03:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 4 = 0.946658334614038, ошибка= 4.625278043107184
 
[17:07:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 5 = 0.943314

In [7]:
da2 = pd.DataFrame( columns=['zone', 'tim1', 'passenger_count', 'trip_distance', 'tolls_amount', 'VendorID_1',
       'VendorID_2', 'payment_type_1', 'payment_type_2', 'payment_type_3',
       'payment_type_4', 'payment_type_5', 'payment_type_6', 'RatecodeID_1',
       'RatecodeID_2', 'RatecodeID_3', 'RatecodeID_4', 'RatecodeID_5',
       'RatecodeID_6', 'time_len'])
da2.set_index(['zone', 'tim1'], inplace = True)

In [8]:
x_array = pd.unique(reg.west)
x_array.sort()
y_array = pd.unique(reg.south)
y_array.sort()
x_array = np.append(x_array, reg.east[-1:].values)
y_array = np.append(y_array, reg.north[-1:].values)

## Для прогнозирования можно использовать информацию, содержащуюся в сырых данных:

средняя длительность поездок

среднее количество пассажиров

среднее расстояние по счётчику

доли поездок, совершаемых по тарифам каждого из типов

доли способов оплаты поездок

средняя стоимость поездок

доли провайдеров данных

In [9]:
da= []
def get(dat):
    data = pd.read_csv(str(dat))
    data = data[data.tpep_pickup_datetime != data.tpep_dropoff_datetime]
    data = data[data.passenger_count != 0]
    data = data[data.trip_distance != 0]
    data = data[data.pickup_longitude >= -74.25559]
    data = data[data.pickup_longitude <= -73.70001]
    data = data[data.pickup_latitude >= 40.49612]
    data = data[data.pickup_latitude <= 40.91553]
    a=stats.binned_statistic_2d(data.pickup_longitude, data.pickup_latitude, None, statistic = "count", bins = [x_array, y_array], expand_binnumbers=True)
    data['zone'] = (a[3][0] - 1) * 50 + (a[3][1]) 
    data = data[data.zone.isin(zone)]    
    b=stats.binned_statistic_2d(data.dropoff_longitude, data.dropoff_latitude, None, statistic = "count", bins = [x_array, y_array], expand_binnumbers=True)
    data['dropoff_zone'] = (b[3][0] - 1) * 50 + (b[3][1])

    data.tpep_pickup_datetime = pd.to_datetime(data.tpep_pickup_datetime)
    data.tpep_dropoff_datetime = pd.to_datetime(data.tpep_dropoff_datetime)
    data['tim1'] = data.tpep_pickup_datetime.map(lambda x: x.replace(minute=0, second=0))
    data['tpep_dropoff_datetime_0'] = data.tpep_dropoff_datetime.map(lambda x: x.replace(minute=0, second=0))
    data['time_len'] = pd.to_datetime(data.tpep_pickup_datetime)
    data.time_len = pd.to_timedelta(data.tpep_dropoff_datetime - data.tpep_pickup_datetime)
    #data.time_len = data.time_len.dt.strftime('%H:%M:%S')
    
    k1=pd.date_range(start=min(data['tim1']), end= max(data['tim1']), freq='H')
    k=len(k1)
    dat_t= pd.DataFrame(columns= ['zone', 'tim1'], index = range(k*len(zone)))
    for i in range(len(zone)):
        dat_t['zone'] [k*i:k*(1+i)] = zone[i]
        dat_t['tim1'] [k*i:k*(1+i)]= k1
    dat_t.tim1 = pd.to_datetime(dat_t.tim1)
    dat_t.set_index(['zone', 'tim1'], inplace = True)    

    passenger_count = data.groupby(['zone', 'tim1'])['passenger_count'].apply(lambda x: x.sum()).reset_index() 
    passenger_count.set_index(['zone', 'tim1'], inplace = True)
    dat_t['passenger_count']= passenger_count['passenger_count']
    trip_distance = data.groupby(['zone', 'tim1'])['trip_distance'].apply(lambda x: x.sum()).reset_index() 
    trip_distance.set_index(['zone', 'tim1'], inplace = True)
    dat_t['trip_distance']= trip_distance['trip_distance']
    tolls_amount = data.groupby(['zone', 'tim1'])['tolls_amount'].apply(lambda x: x.sum()).reset_index() 
    tolls_amount.set_index(['zone', 'tim1'], inplace = True)
    dat_t['tolls_amount']= tolls_amount['tolls_amount']
    
    VendorID_1 = data[data['VendorID'] == 1].groupby(['zone', 'tim1'])['VendorID'].apply(lambda x: x.count()).reset_index().rename(columns={'VendorID': 'VendorID_1'}) 
    VendorID_1.set_index(['zone', 'tim1'], inplace = True)
    dat_t['VendorID_1'] = VendorID_1['VendorID_1']
    VendorID_2 = data[data['VendorID'] == 2].groupby(['zone', 'tim1'])['VendorID'].apply(lambda x: x.count()).reset_index().rename(columns={'VendorID': 'VendorID_2'}) 
    VendorID_2.set_index(['zone', 'tim1'], inplace = True)
    dat_t['VendorID_2'] = VendorID_2['VendorID_2']
    
    payment_type = data.groupby(['zone', 'tim1'])['payment_type'].apply(lambda x: x.value_counts()).reset_index()
    payment_type.set_index(['zone', 'tim1'], inplace = True)
    for i in range(6):
        dat_t['payment_type_'+str(i+1)] = payment_type[payment_type['level_2']== i+1]['payment_type']
    RatecodeID = data.groupby(['zone', 'tim1'])['RatecodeID'].apply(lambda x: x.value_counts()).reset_index()
    RatecodeID.set_index(['zone', 'tim1'], inplace = True)
    for i in range(6):
        dat_t['RatecodeID_'+str(i+1)] = RatecodeID[RatecodeID['level_2']== i+1]['RatecodeID']
    
    dat_t = dat_t.fillna(0)
    
    time_len = data.groupby(['zone', 'tim1'])['time_len'].apply(lambda x: x.sum()).reset_index() 
    time_len.set_index(['zone', 'tim1'], inplace = True)
    dat_t['time_len']= time_len['time_len']
    dat_t['time_len']= dat_t['time_len'].fillna(pd.Timedelta(seconds=0))
    
    return dat_t

In [10]:
%%time
#обработка всех входящих данных для дополнения новыми признаками
for i in range(len(d)):
    d2 = get(d[i])
    #dat[d2.columns] = d2
    da2 = da2.append(d2)
    print(d[i])

yellow_tripdata_2016-06.csv
yellow_tripdata_2016-05.csv
yellow_tripdata_2016-04.csv
yellow_tripdata_2016-03.csv
yellow_tripdata_2016-02.csv
yellow_tripdata_2016-01.csv
yellow_tripdata_2015-12.csv
yellow_tripdata_2015-11.csv
yellow_tripdata_2015-10.csv
yellow_tripdata_2015-09.csv
yellow_tripdata_2015-08.csv
yellow_tripdata_2015-07.csv
yellow_tripdata_2015-06.csv
Wall time: 1h 52min 42s


In [11]:
#Добавляем в датафрейм прошлой недели новые признаки
df = dat.merge(da2, on=['zone', 'tim1'])

In [12]:
df[d2.columns] = df[d2.columns].div(df.val.values,axis=0)

In [13]:
df['time_len']= df['time_len'].fillna(pd.Timedelta(seconds=0))
df['time_len']= df['time_len'].astype(np.int64)
df = df.fillna(0)


In [14]:
df = df.reset_index().set_index('Unnamed: 0') #'Unnamed: 0'

In [15]:
df

,zone,tim1,hour,day,week_day,week,month,year,val,sin1,...,payment_type_4,payment_type_5,payment_type_6,RatecodeID_1,RatecodeID_2,RatecodeID_3,RatecodeID_4,RatecodeID_5,RatecodeID_6,time_len
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1075,2015-06-01 00:00:00,0,1,0,23,6,2015,40,0.037391,...,0.0,0.0,0.0,0.975,0.025,0.0,0.0,0.0,0.0,661375000000
1,1075,2015-06-01 01:00:00,1,1,0,23,6,2015,17,0.074730,...,0.0,0.0,0.0,1.000,0.000,0.0,0.0,0.0,0.0,978411764705
2,1075,2015-06-01 02:00:00,2,1,0,23,6,2015,5,0.111964,...,0.0,0.0,0.0,1.000,0.000,0.0,0.0,0.0,0.0,982000000000
3,1075,2015-06-01 03:00:00,3,1,0,23,6,2015,5,0.149042,...,0.0,0.0,0.0,0.800,0.200,0.0,0.0,0.0,0.0,1041200000000
4,1075,2015-06-01 04:00:00,4,1,0,23,6,2015,8,0.185912,...,0.0,0.0,0.0,1.000,0.000,0.0,0.0,0.0,0.0,923000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969403,2168,2016-06-30 19:00:00,19,30,3,26,6,2016,1,-0.294755,...,0.0,0.0,0.0,0.000,1.000,0.0,0.0,0.0,0.0,3913000000000
969404,2168,2016-06-30 20:00:00,20,30,3,26,6,2016,0,-0.330279,...,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0
969405,2168,2016-06-30 21:00:00,21,30,3,26,6,2016,1,-0.365341,...,0.0,0.0,0.0,0.000,0.000,0.0,0.0,1.0,0.0,20000000000


In [16]:
df.to_csv('dat_c6w6.csv', sep=';', index=True)

In [99]:
%%time
# Разбиваем на выборки, как на прошлой неделе.
train_start = df['tim1'][8*168]
train_stop = "2016-05-01 00:00:00"
test_stop = "2016-05-31 17:00:00"
pred_start = "2016-05-31 23:00:00"
pred_stop = "2016-06-30 17:00:00"
dat_train = df[(df.tim1 >= train_start) & (df.tim1 <= train_stop)]
dat_test = df[(df.tim1 >= train_stop) & (df.tim1 <= pred_start)]
dat_train_pred = df[(df.tim1 >= train_start) & (df.tim1 <= pred_start)]
dat_pred = df[(df.tim1 >= pred_start) & (df.tim1 <= pred_stop)]

Wall time: 13 s


In [18]:
import xgboost as xgb

## Когда вы примете решение остановиться и перестать добавлять признаки, постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза

In [78]:
%%time
# Создаем мега цикл, который посчитает качество модели и остатки в мае, только попробуем другую модель.
X_test = dat_test.copy()
X_test.drop(['zone','tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
#X_test.drop(X_test.loc[:, 'sin1':'cos50'].columns, inplace=True, axis=1)
X_train = dat_train.copy()
X_train.drop(['zone','tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
#X_train.drop(X_train.loc[:, 'sin1':'cos50'].columns, inplace=True, axis=1)

Q_xgb_1 = 0

xgb_cv = []
xgb_model = xgb.XGBRegressor(learning_rate=best_xgb_params[0], n_estimators=best_xgb_params[1], 
                                    reg_lambda=best_xgb_params[2], reg_alpha=best_xgb_params[3], n_jobs=-1)
for i in range(1,7):
    Q_may = 0. # ошибка в мае
    Y_test = dat_test[str(i) + '_ans']
    Y_train = dat_train[str(i) + '_ans']
        
        
        
        #Считаем ошибку по линейной регрессии
    stud = xgb_model.fit(X_train, Y_train)
    
    
    Y_pred = stud.predict(X_test)
    Y_pred[Y_pred < 0] = 0  # All low values set to 0
    Y_score = stud.score(X_test, Y_test)
    Q_may += sum(abs(Y_test-Y_pred))
    Q_xgb_1 += Q_may
    print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_may))
    print(' ')
    
print ('Ошибка данной модели XGB: '+str(1/(102*739*6.)*Q_xgb_1))


[16:25:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 1 = 0.9737830272671941, ошибка= 3.405429689918367
 
[16:29:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 2 = 0.9566960014161033, ошибка= 4.243650059256901
 
[16:32:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 3 = 0.9497669531084257, ошибка= 4.52853451935578
 
[16:35:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 4 = 0.9461304213755628, ошибка= 4.65332115029142
 
[16:38:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 5 = 0.94346304

In [21]:
    
print('Итоговая ошибка за май: ' + str(1/(102*739*6.)*Q_xgb_1))

Итоговая ошибка за май: 26.305868975916937


По итогу значение ошибки не только не уменьшилось, но и возрасло на 0.1. Попробуем подобрать оптимальные параметры, для новых исходых параметров.

In [131]:
%%time
# Создаем мега цикл, который посчитает качество модели и остатки в мае, только попробуем другую модель.
xgb_params = {'learning_rate': [0.1, 0.5, 0.8], 'n_estimators': [200, 500, 700, 1000, 1200, 1500], 
              'reg_alpha': [0.01, 0.1], 'reg_lambda': [0.01, 0.1]}
params = list(product(xgb_params['learning_rate'], xgb_params['n_estimators'], 
                      xgb_params['reg_alpha'], xgb_params['reg_lambda']))
X_test = dat_test.copy()
X_test.drop(['zone','tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
#X_test.drop(X_test.loc[:, 'sin1':'cos50'].columns, inplace=True, axis=1)
X_train = dat_train.copy()
X_train.drop(['zone','tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
#X_train.drop(X_train.loc[:, 'sin1':'cos50'].columns, inplace=True, axis=1)

best_xgb_cv = None
#best_xgb_score = float('inf')
Q_xgb = 40*102*739*6.
best_xgb_params = None
for j in range(len(params)):
    Q_xgb_1 = 0
    param = params[j]
    xgb_cv = []
    xgb_model = xgb.XGBRegressor(learning_rate=param[0], n_estimators=param[1], 
                                    reg_lambda=param[2], reg_alpha=param[3], n_jobs=-1)
    for i in range(1,7):
        Q_may = 0. # ошибка в мае
        Y_test = dat_test[str(i) + '_ans']
        Y_train = dat_train[str(i) + '_ans']
        
        
        
        #Считаем ошибку по линейной регрессии
        stud = xgb_model.fit(X_train, Y_train)
    
    
        Y_pred = stud.predict(X_test)
        Y_pred[Y_pred < 0] = 0  # All low values set to 0
        Y_score = stud.score(X_test, Y_test)
        Q_may += sum(abs(Y_test-Y_pred))
        Q_xgb_1 += Q_may
        print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_may))
        
    
    print ('Ошибка данной модели XGB: '+str(1/(102*739*6.)*Q_xgb_1) + ', при  параметрах: ' +str(param))
    print(' ')
    if Q_xgb_1 < Q_xgb:
        Q_xgb = Q_xgb_1
        best_xgb_cv = xgb_cv
        best_xgb_params = param
print('Лучшие параметры:' +str(best_xgb_params) + ' , ошибка:' + str(1/(102*739*6.)*Q_xgb))
   


[14:09:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 1 = 0.9840534738983562, ошибка= 2.698628357522531
[14:37:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 2 = 0.975608012733073, ошибка= 3.2564685098216106
Ошибка данной модели XGB: 5.955096867344142, при  параметрах: (0.5, 1000, 0.01, 0.01)
 
[15:05:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 1 = 0.9842103136911203, ошибка= 2.6855230195370887
[15:37:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 2 = 0.9758227642931272, ошибка= 3.2350713514995535
Ошибка данной модели XGB: 5.920594371036643, при  параметрах: (0.5, 1200, 0.01, 0.01)


In [129]:
params

[(0.5, 550, 0.01, 0.01), (0.5, 650, 0.01, 0.01), (0.5, 700, 0.01, 0.01)]

In [132]:
print('Лучшие параметры:' +str(best_xgb_params) + ' , ошибка:' + str(1/(102*739*6.)*Q_xgb))

Лучшие параметры:(0.5, 1500, 0.01, 0.01) , ошибка:5.889437512713701


Теперь ошибка за счёт тюнинга и увеличения количества разбиения упала более, чем на 20%!!!

## Переобучите итоговые модели на данных до мая 2016 включительно, постройте прогнозы на июнь для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в уже знакомом вам формате: geoID, histEndDay, histEndHour, step, y

In [133]:
%%time
answer = []
xgb_model = xgb.XGBRegressor(learning_rate=best_xgb_params[0], n_estimators=best_xgb_params[1], 
                                    reg_lambda=best_xgb_params[2], reg_alpha=best_xgb_params[3], n_jobs=-1)
X_test = dat_pred.copy()
X_test.drop(['zone', 'tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train = dat_train_pred.copy()
X_train.drop(['zone', 'tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
Q= 0
for i in range(1,7):
    Q_june = 0. # ошибка в июне
    Y_test = dat_pred[str(i) + '_ans']
    Y_train = dat_train_pred[str(i) + '_ans']
    
    #Считаем ошибку по регрессии
    stud = xgb_model.fit(X_train, Y_train)
    Y_pred = stud.predict(X_test)
    Y_score = stud.score(X_test, Y_test)
    
    Y_pred[Y_pred < 0] = 0  # All low values set to 0
    #models_june[i] = (i, stud)
    Q_june += sum(abs(Y_test-Y_pred))
    Q+= Q_june
    print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_june))
    for  j in range(len(X_test)):
        answer.append([dat_pred.loc[dat_pred.index[j],'zone'].astype(str)+'_'+dat_pred.loc[dat_pred.index[j],'tim1'].strftime('%Y-%m-%d_%H')+'_'+str(i),int(abs(Y_pred[j]))])
    
    print(' ')
print('Итоговая ошибка за июнь ='+str(1/(102*739*6.)*Q))

[18:31:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 1 = 0.9834730990697398, ошибка= 2.4915868467368907
 
[19:25:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 2 = 0.9767490187955091, ошибка= 2.9366686345264315
 
[20:16:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 3 = 0.9728014775480429, ошибка= 3.1769009193663935
 
[21:08:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 4 = 0.9706682457534374, ошибка= 3.288438215836628
 
[22:05:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tочность модели со значением через часа 5 = 0.966

In [134]:
answer

[['1075_2016-05-31_23_1', 20],
 ['1075_2016-06-01_00_1', 12],
 ['1075_2016-06-01_01_1', 4],
 ['1075_2016-06-01_02_1', 4],
 ['1075_2016-06-01_03_1', 3],
 ['1075_2016-06-01_04_1', 7],
 ['1075_2016-06-01_05_1', 24],
 ['1075_2016-06-01_06_1', 37],
 ['1075_2016-06-01_07_1', 54],
 ['1075_2016-06-01_08_1', 66],
 ['1075_2016-06-01_09_1', 53],
 ['1075_2016-06-01_10_1', 64],
 ['1075_2016-06-01_11_1', 90],
 ['1075_2016-06-01_12_1', 84],
 ['1075_2016-06-01_13_1', 108],
 ['1075_2016-06-01_14_1', 104],
 ['1075_2016-06-01_15_1', 79],
 ['1075_2016-06-01_16_1', 86],
 ['1075_2016-06-01_17_1', 108],
 ['1075_2016-06-01_18_1', 112],
 ['1075_2016-06-01_19_1', 107],
 ['1075_2016-06-01_20_1', 107],
 ['1075_2016-06-01_21_1', 105],
 ['1075_2016-06-01_22_1', 60],
 ['1075_2016-06-01_23_1', 37],
 ['1075_2016-06-02_00_1', 18],
 ['1075_2016-06-02_01_1', 6],
 ['1075_2016-06-02_02_1', 5],
 ['1075_2016-06-02_03_1', 2],
 ['1075_2016-06-02_04_1', 9],
 ['1075_2016-06-02_05_1', 25],
 ['1075_2016-06-02_06_1', 42],
 ['1075_2

In [135]:
answer_1 = pd.DataFrame(answer)

In [136]:
answer_1.columns = ['id','y']

In [138]:
answer_1.to_csv('c6w6-ans.csv', sep=',', index=False)

In [137]:
print('Итоговая  ошибка за июнь ='+str(1/(102*739*6.)*Q))

Итоговая  ошибка за июнь =18.977195213899726


В итоге ошибка на kaggle составила 19.62965, неделей раньше была 22.48533. Улучшение - почти 15%!